In [ ]:
# Import necessary libraries
import yfinance as yf
import torch
import torch.nn as nn
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, explained_variance_score
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import ta

# === 0. Set Random Seeds for Reproducibility ===
# This is a critical step to ensure that the results are the same every time the code is run.
seed_value = 42
np.random.seed(seed_value)
torch.manual_seed(seed_value)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value) # if you are using multi-GPU.
    # The two lines below are often needed for full reproducibility on GPUs
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# === Input stock name ===
stock_symbol = input("Enter stock ticker (e.g. TSLA, AAPL, MSFT): ").upper()
start_date = '2015-01-01'
end_date = '2024-12-31'

# 1. Download stock data
print(f"Downloading {stock_symbol} data...")
data = yf.download(stock_symbol, start=start_date, end=end_date)[['Open', 'High', 'Low', 'Close', 'Volume']].dropna()

# FIX: Flatten columns if yfinance returns a MultiIndex
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.get_level_values(0)

print("Data download complete.")


# === 2. Outlier Detection and Removal using Rolling Z-Score ===
print("Identifying and removing outliers...")
data['Daily_Return'] = data['Close'].pct_change()
window = 252
rolling_mean = data['Daily_Return'].rolling(window=window).mean()
rolling_std = data['Daily_Return'].rolling(window=window).std()
data['Z_Score'] = (data['Daily_Return'] - rolling_mean) / rolling_std
threshold = 3.0
outliers = data[data['Z_Score'].abs() > threshold]

if not outliers.empty:
    print(f"Found and removed {len(outliers)} outlier(s).")
    data = data.drop(outliers.index)
else:
    print("No significant outliers found.")

data = data.drop(columns=['Daily_Return', 'Z_Score'])
print("Outlier removal complete.")


# === 3. Feature Engineering (on cleaned data) ===
print("Adding technical indicators...")
data['sma_10'] = ta.trend.SMAIndicator(close=data['Close'].squeeze(), window=10).sma_indicator()
data['ema_10'] = ta.trend.EMAIndicator(close=data['Close'].squeeze(), window=10).ema_indicator()
data['rsi'] = ta.momentum.RSIIndicator(close=data['Close'].squeeze(), window=14).rsi()
data['macd'] = ta.trend.MACD(close=data['Close'].squeeze()).macd()
data['bb_bbm'] = ta.volatility.BollingerBands(close=data['Close'].squeeze()).bollinger_mavg()
data['day_of_week'] = data.index.dayofweek

# --- Add S&P 500 Market Context ---
print("Adding market context (S&P 500)...")
spy_data = yf.download('SPY', start=start_date, end=end_date)
# Flatten columns for SPY data as well for robustness
if isinstance(spy_data.columns, pd.MultiIndex):
    spy_data.columns = spy_data.columns.get_level_values(0)
spy_data['SPY_Return'] = spy_data['Close'].pct_change()
data = data.join(spy_data['SPY_Return'])
# --- End of S&P 500 section ---

data.dropna(inplace=True)
print("Feature engineering complete.")

# Find the index of 'Close' column BEFORE scaling the data
close_price_index = data.columns.get_loc('Close')

# 4. Normalize Data
scaler = MinMaxScaler()
close_price_scaler = MinMaxScaler()
close_price_scaler.fit(data[['Close']])
scaled_data = scaler.fit_transform(data)


# 5. Create sequences
def create_sequences(data, window_size, close_price_idx):
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i])
        y.append(data[i, close_price_idx])
    return np.array(X), np.array(y)

window_size = 60
# Pass the pre-calculated index to the function
X, y = create_sequences(scaled_data, window_size, close_price_index)
print(f"Created {len(X)} sequences with a window size of {window_size}.")

# 6. Split Data (70/30 Train/Test)
train_size = int(0.7 * len(X))
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]
print(f"Data split into {len(X_train)} training samples and {len(X_test)} testing samples.")


# 7. Create DataLoaders
def to_loader(x, y, shuffle_data=True):
    dataset = TensorDataset(torch.tensor(x, dtype=torch.float32),
                             torch.tensor(y, dtype=torch.float32))
    return DataLoader(dataset, batch_size=32, shuffle=shuffle_data)

# Shuffling the training data is a best practice for model generalization
train_loader = to_loader(X_train, y_train, shuffle_data=True)
test_loader = to_loader(X_test, y_test, shuffle_data=False)


# 8. Transformer Model Definition
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=500):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

class StockTransformer(nn.Module):
    def __init__(self, input_dim, model_dim=32, n_heads=2, num_layers=1, output_dim=1):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, model_dim)
        self.pos_enc = PositionalEncoding(model_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=n_heads, batch_first=True, activation='relu')
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(model_dim, output_dim)

    def forward(self, x):
        x = self.input_proj(x)
        x = self.pos_enc(x)
        x = self.transformer(x)
        return self.fc(x[:, -1, :])


# 9. Training Loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
model = StockTransformer(input_dim=X.shape[2]).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

epochs = 50
print(f"Starting training for {epochs} epochs...")
for epoch in range(epochs):
    model.train()
    total_train_loss = 0
    for bx, by in train_loader:
        bx, by = bx.to(device), by.to(device).unsqueeze(1)
        optimizer.zero_grad()
        output = model(bx)
        loss = criterion(output, by)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
    avg_train_loss = total_train_loss / len(train_loader)
    print(f"Epoch {epoch+1:2d}/{epochs} | Train Loss: {avg_train_loss:.5f}")
print("Training complete.")


# 10. Evaluation
print("Evaluating model on the test set...")
model.eval()
preds, actuals = [], []
with torch.no_grad():
    for bx, by in test_loader:
        bx = bx.to(device)
        output = model(bx).cpu().numpy()
        preds.extend(output.flatten())
        actuals.extend(by.numpy())


# 11. Inverse Transform for Plotting and Metrics
preds_reshaped = np.array(preds).reshape(-1, 1)
actuals_reshaped = np.array(actuals).reshape(-1, 1)
scaled_preds = close_price_scaler.inverse_transform(preds_reshaped).flatten()
scaled_actuals = close_price_scaler.inverse_transform(actuals_reshaped).flatten()


# 12. Calculate and Report Metrics (Expanded)
r2 = r2_score(scaled_actuals, scaled_preds)
explained_variance = explained_variance_score(scaled_actuals, scaled_preds)
mae = mean_absolute_error(scaled_actuals, scaled_preds)
rmse = np.sqrt(mean_squared_error(scaled_actuals, scaled_preds))
mape = mean_absolute_percentage_error(scaled_actuals, scaled_preds) * 100
smape_numerator = np.abs(scaled_preds - scaled_actuals)
smape_denominator = (np.abs(scaled_actuals) + np.abs(scaled_preds)) / 2
smape_mask = smape_denominator != 0
smape = np.mean(smape_numerator[smape_mask] / smape_denominator[smape_mask]) * 100

print(f"\n--- Evaluation for {stock_symbol} ---")
print(f"  Goodness of Fit:")
print(f"    R-squared (R²):                 {r2:.4f}")
print(f"    Explained Variance:             {explained_variance:.4f}")
print(f"\n  Average Error:")
print(f"    Mean Absolute Error (MAE):      {mae:.4f} (Error in $)")
print(f"    Root Mean Squared Error (RMSE): {rmse:.4f} (Error in $)")
print(f"    Mean Absolute % Error (MAPE):   {mape:.2f}%")
print(f"    Symmetric MAPE (SMAPE):         {smape:.2f}%")


# 13. Plot Results
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(14, 7))
plt.plot(scaled_actuals, label="Actual Close Price", color='blue', alpha=0.9)
plt.plot(scaled_preds, label="Predicted Close Price", color='red', linestyle='--', alpha=0.8)
plt.title(f"{stock_symbol} Stock Price Prediction (Transformer)", fontsize=16)
plt.xlabel("Time (Test Set Days)", fontsize=12)
plt.ylabel("Close Price (USD)", fontsize=12)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()
